In [ ]:
## Clustering-based Exploration

### Todo: Class for exploration strategy


### Todo: Compare to random walk, lawn mowing


### Todo: Visualization of clustering approach 


### Todo: Pseudocode


### Todo: System-block Diagram